In [11]:
input_paragraph = input(f"Enter a paragraph in Hindi Language:\n")

Enter a paragraph in Hindi Language:
छात्र जीवन में विभिन्न विषयों पर हिंदी निबंध (essay in hindi) लिखने की आवश्यकता होती है। हिंदी निबंध लेखन (essay writing in hindi) के कई फायदे हैं। हिंदी निबंध से किसी विषय से जुड़ी जानकारी को व्यवस्थित रूप देना आ जाता है तथा विचारों को अभिव्यक्त करने का कौशल विकसित होता है। हिंदी निबंध (hindi nibandh) लिखने की गतिविधि से इन विषयों पर छात्रों के ज्ञान के दायरे का विस्तार होता है जो कि शिक्षा के अहम उद्देश्यों में से एक है। हिंदी में निबंध या लेख लिखने से विषय के बारे में समालोचनात्मक दृष्टिकोण विकसित होता है। साथ ही अच्छा हिंदी निबंध


# **Rule Based Approach**

In [ ]:
!pip install inltk



In [ ]:
pip install --upgrade pandas numpy matplotlib networkx nltk inltk indic-nlp-lib


In [ ]:
pip show pandas numpy matplotlib networkx nltk inltk indic-nlp-lib fastai


In [ ]:
!pip install indicnlp


In [ ]:
pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 5.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from nltk.cluster.util import cosine_distance
import string
import math
import operator
import inltk
# from inltk.inltk import setup
import nltk
# from nltk.tokenize import SpaceTokenizer
import re
from matplotlib.font_manager import FontProperties
hindi_font=FontProperties(fname = 'Nirmala-UI.ttf')
import indicnlp
from indicnlp.tokenize import sentence_tokenize
# from inltk.inltk import get_sentence_similarity
# from inltk.inltk import get_sentence_similarity


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Please Load Neccesary stopword File from Dataset Files Folder

In [ ]:
stopwords=pd.read_csv('/kaggle/input/stopwords/StopwordsHindi.csv')
stopwords=list(stopwords['stopwords'])
stopwords.extend(['में','के','है','से'])

In [ ]:
def remove_redundant_sentences(sentences):
    cleaned=[]
    for s in sentences:
        if s in cleaned or s.strip()=='':
            continue
        else:
            cleaned.append(s)
    return cleaned

def clean_corpus(corpus):
    corpus=corpus.replace('।','.')
    corpus=corpus.replace('\xa0','')
    corpus=corpus.replace('\n','')
    corpus=corpus.replace('\r','')
    return corpus

def get_clean_sentences(doc):
    cleaned_doc=clean_corpus(doc)
    sentences=cleaned_doc.split('.')
    sentences=remove_redundant_sentences(sentences)
    return sentences

In [ ]:
patternEnglish = re.compile(r'[[A-Z]|[a-z]]*')
patternNumeric = re.compile(r'.*[0-9]+')
patternBraces=re.compile(r'\[(.*?)\]|\{(.*?)\}|\((.*?)\)')
patternUrl=re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")

/tmp/ipykernel_33/850458489.py:1: FutureWarning: Possible nested set at position 1
  patternEnglish = re.compile(r'[[A-Z]|[a-z]]*')


In [ ]:

def get_len_all_sentences(sentences):
    len_of_all_sent=[len(SpaceTokenizer().tokenize(sentence)) for sentence in sentences]
    max_length=np.max(len_of_all_sent)
    return max_length, len_of_all_sent
def searchEnglishWords(sentence):
    words=SpaceTokenizer().tokenize(sentence.strip())
    words=list(filter(None,words))
    english_words=[word for word in words if patternEnglish.match(word) !=None]
    return len(english_words)/len(words)
def positionScore(total_len):
    mid=(total_len/2)
    positions=(np.arange(total_len,mid,-1)/total_len).tolist()
    if(total_len%2==0):
        positions.extend((np.arange(mid+1,total_len+1,1)/total_len).tolist())
    else:
        positions.extend(np.arange(mid,total_len+1,1).tolist())
    return positions
def numericScore(sentence):
    words=SpaceTokenizer().tokenize(sentence)
    numbers=[1 for word in words if re.match(patternNumeric,word)]
    return np.sum(numbers)/len(words)

def bracesScore(sentence):
    t=re.findall(patternBraces,sentence)
    words=SpaceTokenizer().tokenize(sentence)
    insideTerms=()
    for term in t:
        insideTerms+=term
    insideTerms=[term for term in insideTerms if term !='']
    return (len(words)-len(insideTerms))/len(words)
def url_email_score(sentence):
    words=SpaceTokenizer().tokenize(sentence.strip())
    words=list(filter(None, words))
    urls=[word for word in words if patternUrl.match(word)!=None]
    return len(urls)/len(words)

In [ ]:
def generate_summary_rule_based(clean_sentences,compression_ratio):
    score_table=pd.DataFrame(clean_sentences)
    score_table.columns=['sentence']

#     headlinetokens=SpaceTokenizer().tokenize(sample['highlights_hindi'])
#     heading_titlewords=[ word for word in  headlinetokens if word!= '' and word not in stopwords]
#     score_table['S1']=[titlewordsScore(sentence,heading_titlewords) for sentence in clean_sentences]
    max_length,len_of_all_sent=get_len_all_sentences(clean_sentences)
    score_table['S2']=[x/max_length for x in len_of_all_sent]
    score_table['S3']=[searchEnglishWords(sentence) for sentence in clean_sentences]
    score_table['S4']=[numericScore(sentence) for sentence in clean_sentences]
    score_table['S5']=[bracesScore(sentence) for sentence in clean_sentences]
    score_table['S7']=[url_email_score(sentence) for sentence in clean_sentences]
    #score_table['S8']=positionScore(len(clean_sentences))
    score_table['Total']=score_table['S2']+score_table['S3']+score_table['S4']+score_table['S5']+score_table['S7']
    summary_sent_count=round(compression_ratio*len(clean_sentences))
    summary_sent=list(score_table.sort_values('Total',ascending=False).iloc[0:summary_sent_count]['sentence'])
    summary=""
    for sentence in summary_sent:
        summary+=sentence.strip()+"| "
    return summary

In [ ]:
from nltk.tokenize import SpaceTokenizer


In [ ]:

print("Given Text::",input_paragraph)
clean_sentences=get_clean_sentences(input_paragraph)

summary_rule_based=generate_summary_rule_based(clean_sentences,0.3)
# summary_rule_based_1=generate_summary_rule_based_1(clean_sentences,0.3)

print('Summary obtained from rule based strategy::',summary_rule_based,'\n')
# print('Summary obtained from rule based strategy: ',summary_rule_based_1,'\n')


Given Text:: करमा, इराक (सी. एन. एन.)-महिलाएं इन दिनों अपने चेहरे दिखाने के लिए बहुत डरती हैं और शर्मिंदा होती हैं, न ही वे देर से वेश्यावृत्ति करती हैं, लेकिन वे कभी भी यह नहीं पूछती हैं कि हमें यह रास्ता क्यों अपनाना पड़ा है, न ही हम अपने झंडे का असली नाम इस्तेमाल करना पड़ता है। "तीन की माँ ने अपने बच्चों के लिए मेज पर खाना रखने के लिए अपने शरीरों को बेच दिया, जो एक दिन में 8 डॉलर के बराबर था। सुहा, 37, तीन बच्चों की मां है। वह कहती है कि जब वह घर से निकलती है तो अपने पति को लगता है कि वह घरों की सफाई कर रही है। लोगों को महिलाओं की आलोचना नहीं करनी चाहिए, या उनके बारे में बुरी तरह से बात करनी चाहिए। 37 वर्षीय सुहा कहती है कि आत्महत्या करती है क्योंकि वह इन दिनों उग्रवादियों से बचने के लिए हल्के रंग का स्कार्फ पहनती है, जो खुद को ढकने पर जोर देती है।" वे सभी कहती हैं कि हम अपना रास्ता खो चुके हैं, लेकिन वे कभी नहीं पूछते कि हमें यह रास्ता क्यों अपनाना पड़ा। "तीन की माँ, जो हल्के मेकअप पहनती है, अपने बच्चों के लिए एक सोने का लटकन पहनती
Summary obtained from rule based strategy:: )-महिल

# **TF-IDF Approach**

In [ ]:
def calc_term_frequency_sentence_wise(sentences):
    freq_matrix={}
    for sentence in sentences:
        words=SpaceTokenizer().tokenize(sentence)
        sent_freq_table={}
        for word in words:
            if word in stopwords:
                continue
            if word in sent_freq_table:
                sent_freq_table[word]+=1
            else:
                sent_freq_table[word]=1
        freq_matrix[sentence[:15]]=sent_freq_table
    return freq_matrix

def calc_tf_matrix(freq_matrix):
    for sentence,sent_freq_table in freq_matrix.items():
        tf_sent={}
        sent_len=len(sent_freq_table)
        for word in sent_freq_table:
            sent_freq_table[word]/=sent_len
    return freq_matrix

def calc_sentence_frequency(freq_matrix):
    sent_freq={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        for word,count in freq_matrix_sent.items():
            if word in sent_freq:
                sent_freq[word]+=1
            else:
                sent_freq[word]=1
    return sent_freq

def calc_idf_score(total_sentences,sent_freq,freq_matrix):
    idf={}
    for sentence,freq_matrix_sent in freq_matrix.items():
        idf_sent={}
        for word in freq_matrix_sent:
            idf_sent[word]=math.log10(total_sentences / float(sent_freq[word]))
        idf[sentence]=idf_sent
    return idf

def calc_tf_idf_score(tf,idf):
    tf_idf={}
    for (sentence1,tf_sent),(sentence2,idf_sent) in zip(tf.items(),idf.items()):
        tf_idf_sent={}
        for (word1,tf_score),(word2,idf_score) in zip(tf_sent.items(),idf_sent.items()):
            tf_idf_sent[word1]=tf_score*idf_score
        tf_idf[sentence1]=tf_idf_sent
    return tf_idf

def calc_tf_idf_score_sentence_wise(tf_idf_matrix):
    tf_idf={}
    for sentence,tf_idf_sent in tf_idf_matrix.items():
        tf_idf_sentence=0
        for word,tf_idf_score in tf_idf_sent.items():
            tf_idf_sentence+=tf_idf_score
        tf_idf[sentence]=tf_idf_sentence
    return tf_idf
def get_tf_idf(sentences):
    sent_freq_matrix=calc_term_frequency_sentence_wise(sentences)
    freq_matrix=calc_tf_matrix(sent_freq_matrix)
    total_sentences=len(sentences)
    sent_freq=calc_sentence_frequency(freq_matrix)
    idf=calc_idf_score(total_sentences,sent_freq,freq_matrix)
    tf_idf_matrix=calc_tf_idf_score(freq_matrix,idf)
    tf_idf=calc_tf_idf_score_sentence_wise(tf_idf_matrix)
    return tf_idf

def summarise_tf_idf_sentence_weighting(sentences):
    tf_idf=get_tf_idf(sentences)
    sentences=remove_redundant_sentences(sentences)
    avg_tf_idf_score=average_sentence_value(tf_idf)
    tf_idf_summary,sent_count=generate_summary(avg_tf_idf_score,sentences,tf_idf,15)
    return tf_idf_summary
def average_sentence_value(sentenceValues):
    sum_values=0
    for sent_id in sentenceValues:
        sum_values+=sentenceValues[sent_id]
    sent_count=len(sentenceValues)
    return sum_values/sent_count
def generate_summary(avg_score,sentences,sentenceValue,identifier):
    sentence_count=0
    summary=''
    for sentence in sentences:
        if sentence[:identifier] in sentenceValue and sentenceValue[sentence[:identifier]]>=avg_score:
            summary+=sentence.strip()+"| "
            sentence_count+=1
    return summary,sentence_count

In [ ]:
import pandas as pd

# Assuming df contains your data with 'article_hindi' column and get_clean_sentences(), summarise_tf_idf_sentence_weighting() functions are defined

# Create an empty list to store the cleaned sentences and TF-IDF summaries
# clean = []
# for i in df['article_hindi']:
#     clean.append(get_clean_sentences(i))

clean_sentences=get_clean_sentences(input_paragraph)
# tf_idf_summary = []
# for j in clean:
#     tf_idf_summary.append(summarise_tf_idf_sentence_weighting(j))

tf_idf_summary=summarise_tf_idf_sentence_weighting(clean_sentences)

In [ ]:
print(tf_idf_summary)

करमा, इराक (सी| )-महिलाएं इन दिनों अपने चेहरे दिखाने के लिए बहुत डरती हैं और शर्मिंदा होती हैं, न ही वे देर से वेश्यावृत्ति करती हैं, लेकिन वे कभी भी यह नहीं पूछती हैं कि हमें यह रास्ता क्यों अपनाना पड़ा है, न ही हम अपने झंडे का असली नाम इस्तेमाल करना पड़ता है| "तीन की माँ ने अपने बच्चों के लिए मेज पर खाना रखने के लिए अपने शरीरों को बेच दिया, जो एक दिन में 8 डॉलर के बराबर था| लोगों को महिलाओं की आलोचना नहीं करनी चाहिए, या उनके बारे में बुरी तरह से बात करनी चाहिए| 37 वर्षीय सुहा कहती है कि आत्महत्या करती है क्योंकि वह इन दिनों उग्रवादियों से बचने के लिए हल्के रंग का स्कार्फ पहनती है, जो खुद को ढकने पर जोर देती है| 


# **IndicBART Pipeline**

In [1]:
bart_checkpoint = "ai4bharat/IndicBART"

In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [5]:
tokenizer_bart = AutoTokenizer.from_pretrained(bart_checkpoint,
                                               do_lower_case=False,
                                               use_fast=False,
                                               keep_accents=True)

model_bart = AutoModelForSeq2SeqLM.from_pretrained(bart_checkpoint).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

In [6]:
bos_id = tokenizer_bart._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer_bart._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer_bart._convert_token_to_id_with_added_voc("<pad>")

In [8]:
%%time



inp = tokenizer_bart(input_paragraph, add_special_tokens=False,
                    truncation=True, return_tensors="pt",
                    padding='max_length', max_length=1024)['input_ids'].to(device)

model_output = model_bart.generate(inp, use_cache=True,
                                    num_beams=4,
                                    max_length=70,
                                    min_length=30,
                                    early_stopping=True,
                                    pad_token_id=pad_id,
                                    bos_token_id=bos_id,
                                    eos_token_id=eos_id,
                                    decoder_start_token_id=tokenizer_bart._convert_token_to_id_with_added_voc("<2en>"))

Summary_IndicBART = tokenizer_bart.decode(model_output[0],
                                skip_special_tokens=True,
                                clean_up_tokenization_spaces=False)

print("Summary generated by IndicBART::")
print(Summary_IndicBART)

Summary generated by IndicBART::
करमा, इराक (सी. एन. एन.)-महिलाएं इन दिनों अपने चेहरे दिखाने के लिए बहुत डरती हैं और शर्मिंदा होती हैं, न ही वे देर से वेश्यावृत्ति करती हैं, लेकिन वे कभी भी यह नहीं पूछती हैं कि हमें यह रास्ता क्यों अपनाना पड़ा है, न ही हम अपने झंडे
CPU times: user 3.5 s, sys: 285 ms, total: 3.79 s
Wall time: 5 s


# **mT5 Model**

In [9]:
t5_checkpoint = "csebuetnlp/mT5_multilingual_XLSum"

tokenizer_t5 = AutoTokenizer.from_pretrained(t5_checkpoint)
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(t5_checkpoint)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [12]:
# Tokenize the input text
input_ids = tokenizer_t5(input_paragraph,
                         return_tensors="pt",
                         padding="max_length",
                         truncation=True,
                         max_length=512)["input_ids"]

# Generate the summary
with torch.no_grad():
    model_output = model_t5.generate(input_ids=input_ids,
                                     num_beams=4,
                                     max_length=200,
                                     min_length=30,
                                     no_repeat_ngram_size=2,
                                     early_stopping=True)

# Decode and append the generated summary
decoded_output = tokenizer_t5.decode(model_output[0],
                                     skip_special_tokens=True,
                                     clean_up_tokenization_spaces=False)

# Print the generated summary
print("Generated Summary by mT5 model::", decoded_output)

Generated Summary by mT5 model:: छात्र जीवन में हिंदी में निबंध लिखने की गतिविधि का विस्तार होता है जो कि शिक्षा के अहम उद्देश्यों के दायरे में एक है।
